# 05 - NGram Language Models 
Prepared by Jan Christian Blaise Cruz

DLSU Machine Learning Group

# Preliminaries

First, we'll download the **WikiText-2** dataset to use for this example.

In [ ]:
!wget https://s3.amazonaws.com/research.metamind.io/wikitext/wikitext-2-raw-v1.zip
!unzip wikitext-2-raw-v1.zip && rm wikitext-2-raw-v1.zip

--2020-08-07 03:03:18--  https://s3.amazonaws.com/research.metamind.io/wikitext/wikitext-2-raw-v1.zip
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.107.22
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.107.22|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4721645 (4.5M) [application/zip]
Saving to: ‘wikitext-2-raw-v1.zip’

wikitext-2-raw-v1.z 100%[===================>]   4.50M  --.-KB/s    in 0.1s    

2020-08-07 03:03:18 (37.1 MB/s) - ‘wikitext-2-raw-v1.zip’ saved [4721645/4721645]

Archive:  wikitext-2-raw-v1.zip
   creating: wikitext-2-raw/
  inflating: wikitext-2-raw/wiki.test.raw  
  inflating: wikitext-2-raw/wiki.valid.raw  
  inflating: wikitext-2-raw/wiki.train.raw  


Then we'll import the necessary libraries.

In [ ]:
from tqdm import tqdm
import numpy as np
import pandas as pd

We'll load the dataset and make every word lowercase. WikiText-2 is pre-tokenized so we only have to split by space. We'll also remove blank lines and the headers per article so we'll only be left with the text bodies themselves.

In [ ]:
# Load the dataset
with open('wikitext-2-raw/wiki.train.raw', 'r') as f:
    temp = [l.lower().strip() for l in f]

# Remove blanks and headers, and tokenize
X_train = []
for line in temp:
    if line != '' and not line.startswith('='): X_train.append(line.split())

Here's the first five lines of the dataset tokenized.

In [ ]:
print(X_train[:5])

[['senjō', 'no', 'valkyria', '3', ':', 'unrecorded', 'chronicles', '(', 'japanese', ':', '戦場のヴァルキュリア3', ',', 'lit', '.', 'valkyria', 'of', 'the', 'battlefield', '3', ')', ',', 'commonly', 'referred', 'to', 'as', 'valkyria', 'chronicles', 'iii', 'outside', 'japan', ',', 'is', 'a', 'tactical', 'role', '@-@', 'playing', 'video', 'game', 'developed', 'by', 'sega', 'and', 'media.vision', 'for', 'the', 'playstation', 'portable', '.', 'released', 'in', 'january', '2011', 'in', 'japan', ',', 'it', 'is', 'the', 'third', 'game', 'in', 'the', 'valkyria', 'series', '.', 'employing', 'the', 'same', 'fusion', 'of', 'tactical', 'and', 'real', '@-@', 'time', 'gameplay', 'as', 'its', 'predecessors', ',', 'the', 'story', 'runs', 'parallel', 'to', 'the', 'first', 'game', 'and', 'follows', 'the', '"', 'nameless', '"', ',', 'a', 'penal', 'military', 'unit', 'serving', 'the', 'nation', 'of', 'gallia', 'during', 'the', 'second', 'europan', 'war', 'who', 'perform', 'secret', 'black', 'operations', 'and', 'are

We'll write code to produce ngrams from a series of tokens.

In [ ]:
def make_ngrams(tokens, n):
    temp = []
    for i in range(len(tokens) - n + 1):
        temp.append(tuple(tokens[i:i+n]))
    return temp

Let's test it on the first line of the dataset and make bigrams.

In [ ]:
print(make_ngrams(X_train[0], n=2)[:10])

[('senjō', 'no'), ('no', 'valkyria'), ('valkyria', '3'), ('3', ':'), (':', 'unrecorded'), ('unrecorded', 'chronicles'), ('chronicles', '('), ('(', 'japanese'), ('japanese', ':'), (':', '戦場のヴァルキュリア3')]


Let's turn every line in the dataset into a stream of ngrams, then flatten them into one list.

In [ ]:
n = 3

# Produce then flatten the ngrams
temp = [make_ngrams(t, n) for t in tqdm(X_train)]
ngrams = []
for t in temp: ngrams.extend(t)

print(ngrams[0])

100%|██████████| 17556/17556 [00:00<00:00, 21600.41it/s]

('senjō', 'no', 'valkyria')


Here's the number of ngrams in the entire stream.

In [ ]:
len(ngrams)

1972156

# Co-Occurence Matrix

Next, we'll have to construct the co-occurence matrix. For bigrams, this will be 2d, for trigrams, this will be 3d, and so on. This matrix will stand as our Maximum Likelihood Estimate (MLE) model.

First, let's count the frequency of each unique ngram and the frequency of each word in the entire dataset.


In [ ]:
from collections import Counter

# Count ngram frequencies
ngram_counts = dict(Counter(ngrams))

# Get word frequencies
all_text = []
for t in X_train: all_text.extend(t)
word_counts = dict(Counter(all_text))

Then we'll produce our co-occurence matrix using Pandas MultiIndexing (which makes querying and constructing our matrix much *much* easier.)

In [ ]:
# Get a list of all unique ngrams
ngram_vocab = list(ngram_counts.keys())
ngram_vocab_set = set(ngram_vocab)

# Produce a blank co-occurence matrix
indices = pd.MultiIndex.from_tuples(ngram_vocab)
matrix = pd.Series(np.zeros(len(indices)), index=indices)

We'll then compute for the MLE of each word given its previous $n-1$ words. Again, this is described as:

$$P(W_n | W_{n-1}) = \frac{C( W_{n-1} W_n )}{C(W_{n-1})}$$

Which essentially says that the probability of a word given $n-1$ context words is the frequency of the ngram (context, word) normalized by the frequency of the context. 

In [ ]:
# Copy ngram frequencies into the matrix and normalize
for ngram in tqdm(ngram_vocab):
    matrix[ngram] = (ngram_counts[ngram] / word_counts[ngram[0]])

100%|██████████| 1406280/1406280 [00:29<00:00, 48024.17it/s]


# Using the Matrix

We can then use our co-occurence matrix (our MLE model) to assign probabilities to each sentence. We use the Markov Assumption to calculate this probability:

$$P(W_n | W_1^{n-1}) \approx P(W_n | W^{n-1}_{n - N + 1})$$

Which essentially means that the probability of the sequence so far can be summarized as the probability of the current token and the $n-1$ previous tokens in the sequence, based on the $n$gram used.

We'll code this up by using *log probabilities* to prevent numerical instability. Remember that:

$$p_1 \times p_2 \times ... \times p_n = \exp(\log p_1 + \log p_2 + ... + \log p_n)$$

We'll also add in a small epsilon value of 1e-18 in ``np.log()`` to prevent underflow.

In [ ]:
def get_probability(sentence, n):
    s = sentence.split()
    ngrams = make_ngrams(s, n)
    return np.exp(sum([np.log(matrix[ngram]) if ngram in ngram_vocab_set else 0 for ngram in ngrams]))

Let's test it.

In [ ]:
s = "i went to the kitchen"
o = get_probability(s, n=n)
print(o)

7.061936903452163e-05


To handle unknown tokens, instead of passing 0 outright (which will cause numerical instability), we'll instead pass a very small epsilon value 1e-18.

In [ ]:
s = "i went to the unknowntoken"
o = get_probability(s, n=n)
print(o)

7.061936903452163e-05


# Predict the Next Word

We can use a language model to predict the next word. Essentially, we compute the probability of the entire sequence, then find the maximum probability when multiplied to all possible next tokens. This is called **Greedy Sampling**. In the future we will look at better sampling techniques that produce better sequences.

In [ ]:
def predict_next_token(s, n):
    last_token = s.split()[-1]
    prob = get_probability(s, n)
    return (matrix[last_token] * prob).sort_values().index[-1]

Let's test by predicting the next token.

In [ ]:
predict_next_token('energizer is a brand that manufactures batteries for', n)

('the', 'first')

Another test.

In [ ]:
predict_next_token('my friend and i went to see', n)

('your', 'face')

Let's write a helper function to generate the next $n$ tokens.

In [ ]:
def predict_text(s, n, n_words=10):
    print(s, end=' ')
    for _ in range(n_words):
        ntok = predict_next_token(s, n)
        if type(ntok) is tuple: ntok = ' '.join(ntok)
        print(ntok, end=' ')
        s = s + ' ' + ntok
    print()

And test.

In [ ]:
predict_text('my friend and i went to see', n=n, n_words=10)

my friend and i went to see your face of the united states . the united states . the united states . the united states . the 


In [ ]:
matrix['my']

duty       ,            0.002336
official   authority    0.002336
knowledge  ,            0.002336
magazine   ,            0.002336
sister     ran          0.002336
                          ...   
senior     ...          0.002336
...        enquiry      0.002336
situation  makes        0.002336
window     "            0.002336
tongue     to           0.002336
Length: 343, dtype: float64

# Perplexity

To evaluate language models, we use **Perplexity**, which is the inverse of the probability of the testing set, normalized by the number of words.

First, we'll load the test set of WikiText-2, then preprocess it like the training set.

In [ ]:
# Load the dataset
with open('wikitext-2-raw/wiki.test.raw', 'r') as f:
    temp = [l.lower().strip() for l in f]

# Remove blanks and headers, and tokenize
X_test = []
for line in temp:
    if line != '' and not line.startswith('='): X_test.append(line.split())

Then we code perplexity. Remember:

$$
\begin{aligned}
PP(W) = P(w_1 w_2 ... w_N)^{\frac{1}{N}} \\
= \sqrt[N]{\frac{1}{P(w_1 w_2 ... w_N)}}
\end{aligned}
$$

We'll add a small epsilon value to the denominator to prevent numerical instability.

In [ ]:
def get_perplexity(s, n):
    n_words = len(s.split())
    prob = get_probability(s, n)
    return np.power(1/(prob + 1e-18), 1/n_words)

Here's the perplexity on the training set.

In [ ]:
np.mean([get_perplexity(' '.join(l), n) for l in tqdm(X_train)])

100%|██████████| 17556/17556 [00:42<00:00, 412.64it/s]


6.140256454834692

And the perplexity on the testing set.

In [ ]:
np.mean([get_perplexity(' '.join(l), n) for l in tqdm(X_test)])

100%|██████████| 2183/2183 [00:01<00:00, 1157.03it/s]


2.0040106231448584